### Content-based Recommendation System

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 500

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
groups = pd.read_csv("/dbfs/FileStore/tables/groups_austin.csv")

In [3]:
groups[groups["category.name"] == "food/drink"]

,id,category.name,name,organizer.member_id,topics,description
38,272321,food/drink,Austin Wine Enthusiasts,1953748.0,"[{'urlkey': 'wine', 'name': 'Wine', 'id': 611}...",<p>Welcome to the Austin Wine Enthusiasts Meet...
39,366779,food/drink,Austin's Best - Craft Beer Group,245413651.0,"[{'urlkey': 'singles', 'name': 'Singles', 'id'...",<p>Meet with other local Craft Beer drinkers t...
40,404452,food/drink,"Austin Raw Food Diet, Vegan Potluck and Inform...",3618240.0,"[{'urlkey': 'vegetarian', 'name': 'Vegetarian'...",<p>Are you interested in meeting others who sh...
41,458982,food/drink,Central Texas Foodies,189660465.0,"[{'urlkey': 'wine', 'name': 'Wine', 'id': 611}...",<p>Getting together locally with other people ...
42,488712,food/drink,VegAustin,10775897.0,"[{'urlkey': 'vegetarian', 'name': 'Vegetarian'...",<p>Welcome to VegAustin!&nbsp;We welcome vegan...
43,648177,food/drink,Capital Confectioners Club,1042005.0,"[{'urlkey': 'cakedecorating', 'name': 'Cake De...","<p>Capital Confectioners, a greater Austin Cak..."
44,1255991,food/drink,Fearless Foodies,5183146.0,"[{'urlkey': 'diningout', 'name': 'Dining Out',...",<p>Meetup with other Austin area foodies who s...
45,1279432,food/drink,"Austin Eastside Cooking, Potlucks and Get-toge...",8643052.0,"[{'urlkey': 'weightloss', 'name': 'Weight Loss...",<p>Let's get together and share some good food...
46,1521037,food/drink,Austin Food and Wine Lovers,29885072.0,"[{'urlkey': 'wine', 'name': 'Wine', 'id': 611}...",<p>The Austin Food and Wine Lovers believe tha...
47,1550794,food/drink,VegAustin Potlucks,10775897.0,"[{'urlkey': 'environment', 'name': 'Environmen...",<p>We typically have two potlucks each month a...


In [4]:
groups = groups[["id","category.name","name","organizer.member_id","topics","description"]]

In [5]:
len(groups)

Out[6]: 2891

In [6]:
import json
import yaml
groups["topics"] = groups["topics"].apply(lambda x: yaml.load(x))

type(groups["topics"][0])

/local_disk0/tmp/1572476173384-0/PythonShell.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
 
Out[17]: list

In [7]:
groups = groups["topics"].apply(pd.Series) \
    .merge(groups, right_index = True, left_index = True) \
    .drop(["topics"], axis = 1) \
    .melt(id_vars = ['id', 'category.name','name','organizer.member_id'], value_name = "topics") \
    .drop("variable", axis = 1)

In [8]:
groups = groups["topics"].apply(pd.Series) \
    .merge(groups, right_index = True, left_index = True)

In [9]:
groups = groups[["urlkey","id_y","category.name","name_y","organizer.member_id"]]

In [10]:
groups = groups.rename(columns = {'urlkey':'topic_key','id_y':'group_id','name_y':'group_name'})

In [11]:
groups.head()

,topic_key,group_id,category.name,group_name,organizer.member_id
0,classics,268001,fine arts/culture,The Austin Books and Chatter Group,2850525.0
1,cartoonists,1294488,fine arts/culture,Austin Life Drawing Meetup Group,7518535.0
2,artistsway,1633821,fine arts/culture,Austin Drink-n-Draw,11732501.0
3,musicians,1923631,fine arts/culture,Austin Arts Collective,7601460.0
4,urbanexplor,2295011,fine arts/culture,Urban Sketchers - Austin,13938061.0


In [12]:
groups = groups.dropna(subset = ['topic_key'])

In [13]:
groups[groups["group_id"]==268001]

,topic_key,group_id,category.name,group_name,organizer.member_id
0,classics,268001,fine arts/culture,The Austin Books and Chatter Group,2850525.0
2891,bookclub,268001,fine arts/culture,The Austin Books and Chatter Group,2850525.0
5782,intellectual-discussion,268001,fine arts/culture,The Austin Books and Chatter Group,2850525.0
8673,fiction,268001,fine arts/culture,The Austin Books and Chatter Group,2850525.0
11564,literature,268001,fine arts/culture,The Austin Books and Chatter Group,2850525.0
14455,reading,268001,fine arts/culture,The Austin Books and Chatter Group,2850525.0
17346,novel-reading,268001,fine arts/culture,The Austin Books and Chatter Group,2850525.0


In [14]:
#group_id must also be of category type or else rows will be missing after pivoting table
groups['group_id'] = groups['group_id'].astype('category')

#creating a binary representation utility matrix
temp = groups.drop(['group_name','organizer.member_id'], axis = 1)
temp['dummy'] = 1
temp = pd.pivot_table(temp, index= 'group_id', columns = ['topic_key'], values = 'dummy', fill_value = 0)

In [15]:
#to setup dataframe x that will allow me to map back index to the group title
x = groups.drop_duplicates(subset = 'group_id')
x = x.set_index('group_id')

#joining the category shortname with the pivot table and using get dummies for one hot encoding to have only one level
#of columns
temp = pd.get_dummies(temp.join(x['category.name']), prefix = '', prefix_sep = '')

In [16]:
x.head()

,topic_key,category.name,group_name,organizer.member_id
0,classics,fine arts/culture,The Austin Books and Chatter Group,2850525.0
1,cartoonists,fine arts/culture,Austin Life Drawing Meetup Group,7518535.0
2,artistsway,fine arts/culture,Austin Drink-n-Draw,11732501.0
3,musicians,fine arts/culture,Austin Arts Collective,7601460.0
4,urbanexplor,fine arts/culture,Urban Sketchers - Austin,13938061.0


In [17]:
temp.head()

,1031-real-estate-exchange,10th-amendment,10th-amendment-state-sovereignty,12-step-alternatives,13th-age,15-minute-female-orgasm,2-meter-ham-radio-operators,20-somethings,20s,20s-30s-40s-social-group,20s-30s-social,20s-and-30s,20s-social,24-7-customer-service-support,2600,2d-animation,2nd-american-revolution,30-to-40s-group,30s,30s-40s-social,30s-and-40s,30s-and-40s-fun-loving-desis,3d,3d-animation,3d-artists,3d-gaming,3d-modeling,3d-printing,3dr,4-hour-work-week,4-wheeling-and-camping,4-x-4,4-x-4-trail-riding,40-and-older,40s-and-50s-social-community,420,420-friends,4square,4x4s,5-0-mustangs,5000-year-leap-book-discussions,80s-dancing,80s-movies,80s-music,80s-pop,80s-rock-concerts,80s-rock-music,80smusic,Euro-games,Lesbian,Wakesurfing,_type_-theory,a-capella,a-course-in-miracles-acim,a-course-in-miracles-life,a-course-in-miracles-study-and-sharing-group,a-course-in-miracles-study-group,a-course-in-miracles-thursday-sunday-study-group,a-social-group-for-hiv-positive-people,a-spiritual-path-to-higher-creativity,a11y,abrahamhicks,abstract-art,abundance-and-prosperity,abundance-joy-happiness,abusesurvivors,ac-christians,acapella,acapella-group,acappella,acappella-singing,accelerated-learning-learning-how-to-learn,acceptance,accepting-and-celebrating-a-childfree-life,accessibility,account-based-marketing,accountant,accounting,accounting-saas,accounting-service-consultant,accounting-students,accountsoftware,ace,achieving-your-goals,acim,acim-disappearance-of-the-universe,acim-group,acing-job-interviews-and-job-interview-practice,acoustic-guitar,acoustic-guitar-teachers,acoustic-music,acquired-neuropathy,acro-balancing,acro-yoga,acrobatics,acroyoga,acting,action-sports,active-lifestyle,active-medical-marijuana-patients,active-parenting,active-parents,active-seniors,activedogs,activism,activity-group,activity-monitors-and-devices,acts-of-kindness,acumatica,acupuncture,acustical-music-music-jam-sessions-vinyl-records,addiction,additive-manufacturing,adhd,adhd-support,adminasst,administrative-professionals,adobe,adobe-aem,adobe-cq,adobe-cq5,adobe-creative-suite,adobe-experience-manager,adobe-illustrator,adobe-photoshop,adobe-user-group,adoptee-reunion-support,adoptee-support,adoptees,adoptees-looking-for-reunion,adoption,adoption-reunion-search,adp,adrenal-fatigue,adult-adoptees,adult-amateur-baseball,adult-aspies,adult-baseball,adult-halloween-activities,adult-indigo-crystal-and-rainbow-children,adult-music-lessons,adult-paper-craft-classes,adult-self-defense,adult-social-sports,adult-sports-league,adult-survivors-of-child-abuse,adult-teens-with-asperger-spectrum-disorders,adultindigo,adulting,adults-abused-as-children,adults-adhd-support-group,adults-taking-care-of-aging-parents,adults-with-add-adhd,adults-with-adhd,adults-with-aspergers-syndrome,adults-with-dyspraxia,adults-with-special-needs-disabilities,advaita-vedanta,advanced-algorithms-data-structure-design-pattern,advanced-financial-planning-tax-strategies,advanced-manufacturing-engineering,advanced-photography-workshops,advanced-scuba-diving,advanced-sql-server,adventure,adventure-hiking,adventure-travel,advertising,advertising-and-marketing-digital-and-print,advertising-and-pr,advocacy,adwords,adyashanti,aem,aerial,aerial-cinematography,aerial-performance-art,aerial-photography,aerial-silks,aerial-videography,affenpinscher,affiliate-marketing,affordable-housing,afghanhound,afghanistan-veterans,afh,africa-travel,africa-travel-adventure,african-american,african-american-authors,african-american-families,african-american-professionals,african-american-women,african-authors,african-dance,african-diaspora,african-drumming,african-food,african-men-and-women,african-professionals,african-singles,africanmusic,africans,after-business-hours-networking-meet-up-group,after-death-communications,afterlife,afterlife-and-reincarnation,afternoon-tea-tea-lovers,agda,agents,agile-and-scrum,agile-business-analysts,agile-coaching,agile-leadership,agile-marketing,agile-portfolio-management,agile-

In [18]:
#compute the similarity between groups
from sklearn.metrics.pairwise import cosine_similarity
sim_scores = cosine_similarity(temp)

In [19]:
#reset index to map back group_id recommendation to group names
x = x.reset_index(drop = True)

#created to retrieve idx from input group name of content base function
y = pd.Series(x.index, index = x['group_name'])

In [20]:
y.head(100)

Out[73]: group_name
The Austin Books and Chatter Group 0
Austin Life Drawing Meetup Group 1
Austin Drink-n-Draw 2
Austin Arts Collective 3
Urban Sketchers - Austin 4
Austin Writers Roulette 5
Syncreate: Creativity, Collaboration, & Community in Austin 6
Art Attacks Austin 7
Austin Improv Connection Meetup 8
Austin Street Photography & Street Art Lovers 9
Doodle Dudes USA : AUSTIN, TX 10
Fusebox Technology & Art 11
Bilingual Painting Workshop (Spanish) 12
Friday Night Art Session ‘Vino al Arte’ (Bilingual/Spanish) 13
Austin League of Super Citizens 14
Austin Body Painting Meetup 15
Young Architects/Designers 16
Open Architecture Austin 17
Waterloo Watercolor 18
Every Picture Tells a Story - art exhibition 19
Canvas Stretching 101 20
Architect as Developer Meetup and Happy Hour 21
Austin Drink and Draw :) 22
SCARE for a CURE: Quest Night Pre-Build 23
Austin Voice Over Artists Meetup 24
Midnight Drawing Circle 25
Austin Art & Sound 26
Austin Improv 27
Austin Art Circle 28
Austin Art Lovers 29
 ..
South Austin Soul Supper Potluck - Feed Your Body And Soul! 70
Party of 8 71
Women Wine Down 72
Bittersweet Craft Cocktail Group 73
Austin Burgundy wines Meetup 74
The Chinese Art of Tea 75
Discovering Austin 76
Austin Sake Social 77
ATX Alive 78
Let's Do Fun Sh*t! 79
ATX BBQ Club 80
Austin Wine Tastings 81
Hemp Supper Club 82
Austin Gluten Free Meetup 83
Austin Coffee Culture 84
Table For 4 85
Young, Adventerous, Fun Couples who need friends (DINK) 86
Austin Beer Meetup 87
North Austin Ladies Brunch Bunch 88
Advisory Board Alums 89
Austin Cooking Classes For Fun/Beginners/Intermediate 90
Austin Filipino Meetup 91
Foodie Friend Social 92
Makin' Friends in South Austin 93
The Austin Cocktail Appreciation Society 94
Austin New In Town 🏙 95
Austin Ethnic Eats 96
Austin Mezcal and Tequila Society 97
Let's cook together! social cooking experience 98
Austin Bourbon and Boulders 99
Length: 100, dtype: int64

In [21]:
def content_base(group_name, scores = sim_scores):
    """This function takes in a group name as a string and returns a list of groups that are most similar to the input."""
    indices = list()
    score = list()
    #grabbing the index of the group
    idx = y[group_name]
    
    #find the row of the score matrix using the index and using enumerate to keep track the index of the group similiar
    #to it
    row_score = list(enumerate(scores[idx]))
    
    #sort the row score by similarity score from highest to lowest
    sorted_score = sorted(row_score, key = lambda p: p[1], reverse = True)
    
    for i in sorted_score:
        indices.append(i[0])
        score.append(i[1])
    
    #return sorted_score
    #return the top five most similar groups
    return list(zip(list(x.iloc[indices[1:6],2]), score))

In [22]:
content_base('Austin Drink-n-Draw')

Out[68]: [('Hacks/Hackers Austin', 0.9999999999999998),
 ('Product School Austin', 0.4999999999999999),
 ('Tools for Testing in the DevOps Era - Austin', 0.4999999999999999),
 ('Center for Inquiry - Austin', 0.4999999999999999),
 ('Austin Web Python', 0.408248290463863)]

In [23]:
content_base('Open Architecture Austin')

Out[70]: [('BlockChain Alliance ATX || Speakers Events, Startups &Trends', 1.0),
 ('West Lake Moms Book Club', 0.4677071733467426),
 ('Pittsburgh In Austin', 0.42257712736425834),
 ('Austin - Lesbian Mixers, Events, Activities', 0.38575837490522985),
 ('Networking For Business Growth Austin', 0.3563483225498992)]

In [24]:
content_base('Austin Wine Tastings')

Out[74]: [('Preschooler Play-dates and Family Outings', 1.0),
 ('Understanding our Dimensions & the Branches of Yoga', 0.5698028822981898),
 ('Living Zensideout', 0.5640760748177662),
 ('Learning The Healing Properties Of Plants/Herbs/oils', 0.5454545454545455),
 ('Digital Marketing Austin', 0.5276448530110864)]

In [25]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

In [26]:
tfidf_matrix = tf.fit_transform(groups['description'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
results = {} # dictionary created to store the result in a dictionary format (ID : (Score,item_id))#
for idx, row in groups.iterrows(): #iterates through all the rows

# the below code 'similar_indice' stores similar ids based on cosine similarity. sorts them in ascending order. [:-5:-1] is then used so that the indices with most similarity are got. 0 means no similarity and 1 means perfect similarity#
    similar_indices = cosine_similarities[idx].argsort()[::-1] 
    #similar_indices = cosine_similarities[idx].argsort()

    similar_items = [(cosine_similarities[idx][i], groups['id'][i]) for i in similar_indices]
    results[row['id']] = similar_items[1:]
    
#below code 'function item(id)' returns a row matching the id along with Book Title. Initially it is a dataframe, then we convert it to a list#
def item(id):
    return groups.loc[groups['id'] == id]['name'].tolist()[0]
def recommend(id, num):
    if (num == 0):
        print("Unable to recommend any group as you have not chosen the number of groups to be recommended")
    elif (num==1):
        print("Recommending " + str(num) + " group similar to " + item(id))
        
    else :
        print("Recommending " + str(num) + " groups similar to " + item(id))
        
    print("----------------------------------------------------------")
    recs = results[id][:num]
    for rec in recs:
        print("You may also like to visit events of the group: " + item(rec[1]) + " (score:" + str(rec[0]) + ")")

#the first argument in the below function to be passed is the id of the book, second argument is the number of books you want to be recommended#
recommend(272321,5)

Recommending 5 groups similar to Austin Wine Enthusiasts
----------------------------------------------------------
You may also like to visit events of the group: Austin Wine Tastings (score:0.09834056270564631)
You may also like to visit events of the group: Austin Texas Wine Society Meetup Group (score:0.08085183735861684)
You may also like to visit events of the group: Jazz Theory and it's Practical Application. All levels. (score:0.0783912785206729)
You may also like to visit events of the group: Austin Burgundy wines Meetup (score:0.07026972824622615)
You may also like to visit events of the group: Austin Food and Wine Lovers (score:0.05532172340997247)

In [27]:
test_list = np.array([1,3,4,5,6,8,1,.1,67,86,94,-17])

In [28]:
test_list.argsort()

Out[33]: array([11, 7, 0, 6, 1, 2, 3, 4, 5, 8, 9, 10])